In [4]:
myPath = '/Users/leguillou/WORK/Developpement/'

## Packages

In [14]:
import numpy as np
import netCDF4 as nc
import os,fnmatch
import xarray as xr
import pandas as pd 

## Parameters

In [21]:
dir_data = myPath + 'DATA/eNATL60/merged_outputs/'
file_data = 'eNATL60_GULFSTREAM_SSH.1h_y2009m10d01_y2009m12d31.nc'
name_time = 'time_counter'
name_lon = 'nav_lon'
name_lat = 'nav_lat'
name_var = 'sossheig'

In [22]:
dir_out = myPath + 'DATA/eNATL60/hourly_outputs/'
prefixe_outputs = 'eNATL60_GULFSTREAM_SSH'

## Read data

In [54]:
ds = xr.open_mfdataset(dir_data+file_data)
times = ds[name_time]
lon = ds[name_lon]
lat = ds[name_lat]
var = ds[name_var]
NT,NY,NX = var.shape

## Split data by time

In [56]:
for t in range(NT):
    ts = pd.to_datetime(str(times[t].values)) 
    d = ts.strftime('y%Ym%md%dh%H')
    output_name = dir_out+prefixe_outputs+'_'+d+'.nc'
    # Write output
    ncout = nc.Dataset(output_name, 'w', format='NETCDF4_CLASSIC')
    ncout.createDimension('time', 1)
    ncout.createDimension('y', NY)
    ncout.createDimension('x', NX)
    nctime = ncout.createVariable(name_time, 'f', ('time'))
    nclon = ncout.createVariable(name_lon, 'f', ('y','x'))
    nclat = ncout.createVariable(name_lat, 'f', ('y','x'))
    ncvar = ncout.createVariable(name_var, 'f', ('time','y','x'))
    nctime[:] = times[t].astype('datetime64[s]').astype('int')
    nctime.units = 'UNIX timestamp'
    nclon[:,:] = lon
    nclat[:,:] = lat
    ncvar[:,:,:] = var[t,:,:]
    ncout.close()
    
    